In [ ]:
import cv2
import numpy as np
from ipywidgets import *
import matplotlib.pyplot as plt
import math
from math import atan2, cos, sin, sqrt, pi
from sympy import Point, Polygon, Line, Segment

#custom libraries
from TrajectoryPlanner import Node, Planner
from Vision import Vision

In [ ]:
#
min_blue = np.array([70, 80, 0])
min_red = np.array([70, 80, 0])

max_blue = np.array([70, 80, 0])
max_red = np.array([70, 80, 0])

min_green = np.array([21, 0, 0])
max_green = np.array([80, 255, 216])

x = 0 #tymio initial position
y = 0 #tymio initial position
angle = 0 #tymio angle
radius = 0 #tymio radius

goal_x = 0 #goal position
goal_y = 0 #goal position

vertices = [] #vertices



"""
vision = Vision()
x_center,y_center, goal_detected = vision.get_goal_position(frame)
center, radius, angle, thymio_detected = vision.get_thymio_position(frame,(0,0),0)
print("Position: {0} , goal: {1},{2}".format(center,x_center,y_center))
vertices = vision.get_obstacle_position(frame,radius,min_green,max_green)[0]"""

In [ ]:
#Planning
planner = Planner()
planner.setup(vertices,[0,0],[350,280])
planner.plot()

In [ ]:
x = 0
y = 0
while not goal:
    x,y,angle = estimate_position()
    if not obstacle:
        controller_trajectory(x,y,angle,path)
    else:
        controller_obstacle_avoid(x,y,angle,path)
        if not obstacle:
            path = replan()
        